学習用コード．

In [1]:
from google.colab import drive
drive.mount('/content/drive')
rundir = "/content/drive/MyDrive/infalpha/"

In [2]:
import csv
import numpy as np
import scipy as sp
import sklearn as sk
import matplotlib as mpl
import matplotlib.pylab as plt
import tensorflow as tf
import keras
import random
from keras.models import Model
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, Flatten, Dropout, Activation, BatchNormalization
from sklearn.model_selection import train_test_split
from PIL import Image
from keras.utils.np_utils import to_categorical
from datetime import date, timedelta
from tensorflow.python.keras.models import load_model


In [ ]:
#1461日分のデータがある
#データ数を減らした状態で扱う．(1458 - dec_cnt) 個．
dec_cnt = 1420 #GitHubにアップロードしているデータは50件程度しかないため．実際の学習の際はdec_cntを1200程度にしていた．
start_date = 0 ##############学習時はこの値を適当に変更##############
#前3日分+当日の降水量を用いて予測→4日分でひとまとまりのイメージ
#1458個のデータにできる

#快晴などの天気のデータを「晴れ」「雨」「曇り」「その他」に分類．雪などは雨．
def get_val(v):
    #快晴，晴れ
    if v==1 or v==2:
        return 0
    #雨，みぞれ，雪，あられ，ひょう，雷，霧，霧雨
    if 8<=v and v<=15:
        return 1
    #曇り，薄曇り
    if v==3 or v==4:
        return 2
    return 3


x = [[] for i in range(start_date, 1458 - dec_cnt + start_date)]
y = [0 for i in range(start_date, 1458 - dec_cnt + start_date)]
for toshi in ['tokyo', 'nagoya', 'osaka', 'nigata', 'sendai', 'kagoshima']:
    with open(rundir + toshi + '2.csv', 'r') as f:
        read = csv.reader(f)
        l = [row for row in read]
        for i in range(1 + start_date,1459 - dec_cnt + start_date):
            #i, ..., i+2 行目をxに用いる
            for j in range(3):
                x[i-1-start_date] += l[i+j]
            #i+3行目の天気のデータがyに．
            #晴れ系統，雨雪系統，曇り系統，その他　に分類
            if toshi == 'tokyo':
                y[i-1-start_date] = get_val(int(l[i+3][4]))
for toshi in ['choshi', 'kumagaya', 'koufu', 'yokohama']:
    with open(rundir + toshi + '2.csv', 'r') as f:
        read = csv.reader(f)
        l = [row for row in read]
        #l[i][5]を全て消す
        for i in range(len(l)):
            l[i].pop(5)
        for i in range(1 + start_date,1459 - dec_cnt + start_date):
            #i, ..., i+2 行目をxに用いる
            for j in range(3):
                x[i-1-start_date] += l[i+j]
for i in range(len(x)):
    for j in range(len(x[0])):
        if x[i][j] == '':
            x[i][j] = -100. # 欠損データは-100を割り当てておく．
        else:
            x[i][j] = np.float32(x[i][j])
    y[i] = int(y[i])

for i in range(len(x)):
    x[i] = np.array(x[i])

x = np.array(x)
y = np.array(y)
y = to_categorical(y) #ワンホットベクトルに

In [4]:
#訓練データの準備

def get_imgs_array(name, size):
    with open(rundir + name) as f:
        return (np.array(list(map(int, f.readlines()[0].split())))).reshape(size)

#画像たちをどうにかする場所
#chart : 450 * 600 で RGB
#rader : 520 * 692  で RGB
#satellite : 520 * 692 で RGB

#1/3からスタートなので注意．
#1458個データがある
ALL_DATA = [i for i in range(start_date, 1458 - dec_cnt + start_date)]
TRAIN_DATA, TEST_DATA = train_test_split(ALL_DATA, test_size=0.05, random_state=42)

#テストデータは使う時にやる．RAMを確保したいのでとりあえず空に．
model = []
x_test = []
y_test = []
c_18_test = []
r_18_test = []

#ここから訓練データ
x_train = []
y_train = []
c_18_train = []
r_18_train = []

for i in TRAIN_DATA:
    y_train.append(y[i-start_date])
    x_train.append(x[i-start_date])
    c_18_train.append(get_imgs_array('chart/' + format(i,'#4') + '-18.txt', (450, 600, 3))/255)
    r_18_train.append(get_imgs_array('rader/' + format(i,'#4') + '-18.txt', (520, 692, 3))/255)
x_train = np.array(x_train)
c_18_train = np.array(c_18_train)
r_18_train = np.array(r_18_train)
y_train = np.array(y_train)

In [ ]:
#0から学習する場合 - 1/2

input_dim = 3 * (10 * 7 - 4) #10都市，3日分のデータを用いる．雲量を使わない箇所が4つあるので注意
output_dim = 4 #晴れ，雨，曇り，その他　に分類．

#画像関連
#chart : 600 * 450 で RGB
#rader : 692  * 520 で RGB
#satellite : 692  * 520 で RGB

inputs = Input(shape=(input_dim,), name="Input")
middle1 = Dense(30, activation="linear", name="1st")(inputs)
normal_1 = BatchNormalization()(middle1)
act_1 = Activation('relu')(normal_1)
middle2 = Dense(30, activation="linear", name="2nd")(act_1)
normal_2 = BatchNormalization()(middle2)
act_2 = Activation('relu')(normal_2)

#画像の方の入力口
inputs_c18 = Input(shape=(450, 600, 3), name="Input_c18")
conv_c18 = Conv2D(2,(9, 9),activation="linear", name="conv_c18")(inputs_c18)
normalize_c = BatchNormalization()(conv_c18)
act_c = Activation('relu')(normalize_c)
maxpooling_c = MaxPooling2D(pool_size=(2, 2), name="c18_pooling")(act_c)
conv_c18_2 = Conv2D(2,(9, 9),activation="linear", name="conv_c18_2")(maxpooling_c)
normalize_c_2 = BatchNormalization()(conv_c18_2)
act_c_2 = Activation('relu')(normalize_c_2)
maxpooling_c_2 = MaxPooling2D(pool_size=(2, 2), name="c18_pooling_2")(act_c_2)
flat_c18 = Flatten(name="flat_c18")(maxpooling_c_2)
last_c = Dense(11, activation='relu', name='last_c')(flat_c18)


inputs_r18 = Input(shape=(520, 692, 3), name="Input_r18")
conv_r18 = Conv2D(2,(9,9),activation="linear", name="conv_r18")(inputs_r18)
normalize_r = BatchNormalization()(conv_r18)
act_r = Activation('relu')(normalize_r)
maxpooling_r = MaxPooling2D(pool_size=(2, 2), name="r18_pooling")(act_r)
conv_r18_2 = Conv2D(2,(9,9),activation="linear", name="conv_r18_2")(maxpooling_r)
normalize_r_2 = BatchNormalization()(conv_r18_2)
act_r_2 = Activation('relu')(normalize_r_2)
maxpooling_r_2 = MaxPooling2D(pool_size=(2, 2), name="r18_pooling_2")(act_r_2)
flat_r18 = Flatten(name="flat_r18")(maxpooling_r_2)
last_r = Dense(11, activation='relu', name='last_r')(flat_r18)


combined = tf.keras.layers.concatenate([act_2, last_c, last_r])
outputs = Dense(output_dim, activation='softmax', name="3rd")(combined)


model = keras.Model(inputs=[inputs, inputs_c18, inputs_r18], outputs=outputs, name="weather_forecast")
model.summary()
keras.utils.vis_utils.plot_model(model)

In [20]:
#0から学習する場合 - 2/2
model.compile(
    loss=keras.losses.CategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(),
    metrics=["accuracy"],
)

In [5]:
#追加学習の場合 - 1/1
model = keras.models.load_model(rundir + 'my_model.h5') #ファイル名は適宜．

In [ ]:
#学習
#RAMに載せる
epochs = 10
#batch_size=32なのは，アップロードしているデータが50件程度しかないため．実際の学習の際は64にしていた．
history = model.fit([x_train, c_18_train, r_18_train], y_train, batch_size=32, epochs=epochs, verbose=1, validation_split=0.2)

In [7]:
model.save(rundir + 'my_model_2.h5') #ファイル名は適宜．